In [ ]:
# Import 1
# in your notebook cell
import sys
# path relative to your notebook
sys.path.append("/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/src")
# import as usual
import functions

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Import 2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read file in and drop first two rows
# First two rows seem to make like no sense with the rest of this data set
df = functions.read_file('/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/data/dc_crime_add_vars.csv')
df = df.drop(['Unnamed: 0', 'X'], axis=1)
df.head()

In [ ]:
# Describing the Dataset 1
df.describe()

In [ ]:
# Describing the Dataset 2
df.info()

In [ ]:
# Describing the Dataset 3
df.index

In [ ]:
# Describing the Dataset 4
df.columns

In [ ]:
# Describing the Dataset 5
missing_v = df.isnull().sum()
missing_v

In [ ]:
# Describing the Dataset 6
# Went from 342867 to 325340 = 17527 removed which is roughly 5% of the dataset 
df = functions.drop_rows_with_missing_data(df)
df.info()

In [ ]:
# Describing the Dataset 7
functions.describe_dataframe(df)

In [ ]:
# Describing the Dataset 8
functions.get_column_features(df)

In [ ]:
# Slice df to only include the offense of homicide
# Convert datetimes
# Make new start and end hour column
df_2 = df[df['OFFENSE']=='HOMICIDE']
df_2['START_DATE'] = pd.to_datetime(df_2['START_DATE'])
df_2['END_DATE'] = pd.to_datetime(df_2['END_DATE'])
df_2['start_hour'] = df_2['START_DATE'].dt.hour
df_2['end_hour'] = df_2['START_DATE'].dt.hour
df_2 = df_2.drop('hour', axis=1)
df_2.head()

In [ ]:
# Dedup'd for final clean df
# 1100 rows now
hom_df = df_2

# Slicing df one last time to get only my desired columns
hom_df = hom_df[['REPORT_DAT',
                'SHIFT',
                'OFFENSE',
                'METHOD',
                'BLOCK',
                'DISTRICT',
                'PSA',
                'WARD',
                'ANC',
                'NEIGHBORHOOD_CLUSTER',
                'BLOCK_GROUP',
                'VOTING_PRECINCT',
                'START_DATE',
                'END_DATE',
                'XBLOCK',
                'YBLOCK',
                'date',
                'year',
                'start_hour',
                'end_hour',
                'crimetype']]

#Show that it worked
hom_df.columns

In [ ]:
# Have homicides increased over this decade?
# Line Chart

# Filter the DataFrame for homicides
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by year and count the number of homicides per year
homicides_by_year = homicides_df.groupby('year').size()


In [ ]:
# Which districts or neighborhoods had the highest number of homicides?
# Bar Chart

# Filter the DataFrame for homicides
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by district/neighborhood and count the number of homicides
homicides_by_location = homicides_df['DISTRICT'].value_counts().sort_values()

In [ ]:
# Is there any correlation between the time of the day and the occurrence of homicides?
# Heat Map

# Extract the hour from the 'REPORT_DAT' column
df['Time_of_Day'] = pd.to_datetime(df['REPORT_DAT']).dt.hour

# Create a binary(dummy) column for the cat columns
df['Homicide'] = df['OFFENSE'].apply(lambda x: 1 if 'HOMICIDE' in x else 0)
df['Shift_Evening'] = df['SHIFT'].apply(lambda x: 1 if 'EVENING' in x else 0)
df['Shift_Midnight'] = df['SHIFT'].apply(lambda x: 1 if 'MIDNIGHT' in x else 0)

# Select the columns of interest
correlation_df= df[['Time_of_Day', 'Homicide', 'Shift_Evening', 'Shift_Midnight']]
corr_matrix = correlation_df.corr()

In [ ]:
# Are homicides more likely to occur with a gun or a knife? 
# Stacked Plot

# Select list of violent offenses
offenses = ['HOMICIDE', 'ROBBERY', 'ASSAULT W/DANGEROUS WEAPON', 'BURGLARY']

# Filter the DataFrame
filtered_df = df[offenses]

# Group the data by offense and method, and count
# .size()is used after grouping the data to count the number of occurrences 
# for each group. It returns a Series or DataFrame with the counts.
# .unstack() is applied to a hierarchical index (such as the result of a groupby operation) 
# and reshapes the data by "unstacking" one level of the index to create columns.
grouped_data = filtered_df.groupby(['OFFENSE', 'METHOD']).size().unstack()

# Calculate the proportions for each method within each offense
grouped_data_proportions = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100  # Multiply by 100 to get percentages

In [ ]:
# Extra: Folium Experiment?
# Show a heat map of the highest number of homicides, in a specfic area, using leaflet.js